# demonstrate geolocation with geocode API

In [1]:
from text2graph.geolocation.geocode import get_gps
from text2graph.schema import RateLimitedClient


lat, lon, url = await get_gps("little pybus bay", client=RateLimitedClient(interval=1., count=1, timeout=30))
print(lat, lon, url)

57.2436111 -134.1333333 https://geocode.maps.co/search?&q=little pybus bay


In [2]:
# # get text from hybrid endpoint

# import os
# import requests
# 
# 
# ASK_XDD_APIKEY = os.getenv("ASK_XDD_APIKEY")
# ENDPOINT = "http://cosmos0001.chtc.wisc.edu:4502/hybrid"
# 
# headers = {"Content-Type": "application/json", "Api-Key": ASK_XDD_APIKEY}
# data = {
#     "topic": "criticalmaas",
#     "question": "Little Pybus Bay Stratigraphy",
#     "top_k": 10,
#     "temperature": 1.0,
# }
# 
# response = requests.post(ENDPOINT, headers=headers, json=data)
# text = response.json()[2]['text_content']
# text

In [1]:
from text2graph.llm import ask_llm, OpenSourceModel, OpenAIModel
from text2graph.prompt import PromptHandlerV3
from text2graph.alignment import AlignmentHandler

text = 'In the county of Annapolis, Nova Scotia, and in the vicinity of Nictaux Silurian strata occur, including the Nictaux iron ore beds and the Torbrook sandstone formation, whilst near Kentville the Kentville formation is seen, as well as on Angus Brook, in the Gaspereau Valley, also at New Canaan, with Dictyonema websteTi Dawson, and at Wolfville, in King\'s County, where coralline limestones, red and green graptolitic slates, and other strata, at times highly cleaved, squeezed, and metamorphosed, form conspicuous ridges and constitute the oldest sedimentaries in the vicinity of the Bay of Fundy and the Blomidon region in the "land of Evangeline.\'\''

# model = OpenSourceModel("mixtral")
model = OpenAIModel.GPT4T
results = await ask_llm(
        text=text,
        prompt_handler=PromptHandlerV3(),
        model=model,
        temperature=0.0,
        to_triplets=True,
        alignment_handler=AlignmentHandler.load(
            "data/known_entity_embeddings/all-MiniLM-L6-v2"
        ),
    )

results

GraphOutput(triplets=[RelationshipTriplet(subject=Location(name='Annapolis, Nova Scotia', lat='44.7064932', lon='-65.2498142', provenance=Provenance(id=UUID('a1d81649-1556-423c-a7e4-98e1915e62fd'), source_name='GeocodeAPI', source_url='https://geocode.maps.co/search?&q=Annapolis, Nova Scotia', source_version=None, requested=datetime.datetime(2024, 4, 23, 15, 28, 30, 771453), additional_values={}, previous=Provenance(id=UUID('0d9e3086-cfa0-4fc5-96da-52f95569b0f3'), source_name='OpenAIModel', source_url=None, source_version='gpt-4-turbo-preview', requested=datetime.datetime(2024, 4, 23, 20, 28, 13, 651180, tzinfo=datetime.timezone.utc), additional_values={'temperature': 0.0, 'prompt': 'v3', 'doc_ids': []}, previous=None))), predicate='occur', object=Stratigraphy(strat_name='Nictaux', strat_name_long=None, rank=None, strat_name_id=None, concept_id=None, bed=None, bed_id=None, mbr=None, mbr_id=None, fm=None, fm_id=None, subgp=None, subgp_id=None, gp=None, gp_id=None, sgp=None, sgp_id=None,

In [9]:
strat_name = results.triplets[1].object
location = results.triplets[1].subject

print(f"{strat_name.strat_name_long=}")
print(f"{location.name=}")
print(f"{location.lat=}")
print(f"{location.lon=}")

strat_name.strat_name_long='Torbrook Formation'
location.name='Annapolis, Nova Scotia'
location.lat='44.7064932'
location.lon='-65.2498142'
